<a href="https://colab.research.google.com/github/stbalaji/AIForBusiness/blob/main/05%20Generative%20AI/AICH_AIB_M4_GenAI_Enterprise_RAG_Example02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain sentence-transformers chromadb
# faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 86.2 MB/s eta 0:

In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings  # will not download but remain in cloud

from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub

from langchain.chains import RetrievalQA, LLMChain

## Setup HuggingFace Access Token

- Log in to [HuggingFace.co](https://huggingface.co/)
- Click on your profile icon at the top-right corner, then choose [“Settings.”](https://huggingface.co/settings/)
- In the left sidebar, navigate to [“Access Token”](https://huggingface.co/settings/tokens)
- Generate a new access token, assigning it the “write” role.

In [ ]:
#Hugging Face : Need Access Token
import os
from getpass import getpass

HF_TOKEN = getpass("HF Token:")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

HF Token:··········


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
WEBSITE_URL = "https://tarunjain.netlify.app/"

In [ ]:
loader = WebBaseLoader(WEBSITE_URL)
loader.requests_per_second = 1
docs = loader.aload()

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  7.04it/s]


In [ ]:
docs

[Document(page_content='\n\n\n\n\nTarun Jain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nAbout Me\nEvents\nAchievements\nProjects\nWork\nConnect\n\nResume\n\n\nAI with Tarun\n\n\n\n\n\n\n\n\n\n\n\n\nIt\'s Me...  Tarun Jain\n\n DevRel @AI Planet🥑 ||  AI with Tarun- YouTube🤗||  GDE in ML🚀 ||  GSoC\'23 caMicroscope\nKnow more\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAbout Me\n\n\r\n                Hello! My name is Tarun R Jain. I\'m a passionate coder with expertise in Machine Learning, Image Processing, and Deep Learning. I\'ve published over 80 blog articles documenting my coding journey, and I\'m actively involved in various communities, including Hugging Face-Keras Working Group, Deep Learning AI- Bangalore Ambassador, TensorFlow User Group Bangalore- Assistant Organizer and Geeksforgeeks- Technical content writer.\r\n              \n\r\n                What I love most about coding is the ability to create something new and solve complex problems. I\'m constantly learning and experim

In [ ]:
# Break Text into Components - Text Splitter. To avoid repetation etc..
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=20) # Chunk Overlap is important. Begining a Chunk context is also there in the next chuck.
chunks = text_splitter.split_documents(docs)

In [ ]:
len(chunks)

32

In [ ]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name="BAAI/bge-base-en-v1.5"
)

In [ ]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
vectorstore

In [ ]:
llm = HuggingFaceHub(
    repo_id="huggingfaceh4/zephyr-7b-alpha",
    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
query = "Where does Tarun work?"
search = vectorstore.similarity_search(query)

# Step 1 of RAG : Retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr", #similarity
    search_kwargs={'k': 2}           # How many relevant documents we want to retrive
)

In [ ]:
query = "Who is DevRel @AI Planet?"
doc_rels = retriever.get_relevant_documents(query)
doc_rels

[Document(page_content="Tarun Jain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nAbout Me\nEvents\nAchievements\nProjects\nWork\nConnect\n\nResume\n\n\nAI with Tarun\n\n\n\n\n\n\n\n\n\n\n\n\nIt's Me...  Tarun Jain\n\n DevRel @AI Planet🥑 ||  AI with Tarun- YouTube🤗||  GDE in ML🚀 ||  GSoC'23 caMicroscope\nKnow more", metadata={'language': 'pt-BR', 'source': 'https://tarunjain.netlify.app/', 'title': 'Tarun Jain'}),
 Document(page_content='DevFest Kathmandu, Nepal\n\r\n                 Got invited at Kathmandu Nepal to speak on Retrieval Augmented Generation and Fine Tuning for large language models and code demo', metadata={'language': 'pt-BR', 'source': 'https://tarunjain.netlify.app/', 'title': 'Tarun Jain'})]

In [ ]:
doc_rels[0]

Document(page_content="Tarun Jain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nAbout Me\nEvents\nAchievements\nProjects\nWork\nConnect\n\nResume\n\n\nAI with Tarun\n\n\n\n\n\n\n\n\n\n\n\n\nIt's Me...  Tarun Jain\n\n DevRel @AI Planet🥑 ||  AI with Tarun- YouTube🤗||  GDE in ML🚀 ||  GSoC'23 caMicroscope\nKnow more", metadata={'language': 'pt-BR', 'source': 'https://tarunjain.netlify.app/', 'title': 'Tarun Jain'})

In [ ]:
doc_rels[1]

Document(page_content='DevFest Kathmandu, Nepal\n\r\n                 Got invited at Kathmandu Nepal to speak on Retrieval Augmented Generation and Fine Tuning for large language models and code demo', metadata={'language': 'pt-BR', 'source': 'https://tarunjain.netlify.app/', 'title': 'Tarun Jain'})

In [ ]:
llm = HuggingFaceHub(
    repo_id="huggingfaceh4/zephyr-7b-alpha",
    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


# Step 2 of RAG : Augment

In [ ]:
query = "Name the projects Tarun has worked on?"

# System Prompt and User Prompt

prompt = f"""
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

# Step 3 of RAG : Generate

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever)  #Chain_Type = Stuff or refine etc


In [ ]:
response = qa.invoke(prompt)

In [ ]:
response

{'query': '\n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n Name the projects Tarun has worked on?\n </s>\n <|assistant|>\n',
 'result': '\n\nOriginal answer: Tarun has worked on various projects, including GDE in ML, Project Ca Microscope, AI with Tarun, and DevRel at AI Planet.\n\nRefined answer: In addition to the projects mentioned earlier, Tarun has also worked on MangaPi, a software that converts manga pages into digital format. He has also contributed to a research paper on hyperspectral image compression using MATLAB (Hardware), published in the International Journal of Electrical and Computer Engineering. To stay updated on his latest tutorials and resources, you can subscribe to his YouTube channel AI with Tarun.'}

In [ ]:
print(response['result'])



Original answer: Tarun has worked on various projects, including GDE in ML, Project Ca Microscope, AI with Tarun, and DevRel at AI Planet.

Refined answer: In addition to the projects mentioned earlier, Tarun has also worked on MangaPi, a software that converts manga pages into digital format. He has also contributed to a research paper on hyperspectral image compression using MATLAB (Hardware), published in the International Journal of Electrical and Computer Engineering. To stay updated on his latest tutorials and resources, you can subscribe to his YouTube channel AI with Tarun.


In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
template = """
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

In [ ]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

TypeError: unsupported operand type(s) for |: 'dict' and 'str'

In [ ]:
response = rag_chain.invoke("Name the projects Tarun has worked on?")

NameError: name 'rag_chain' is not defined

In [ ]:
print(response)